In [1]:
from sklearn.decomposition import PCA
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
import math
import torch.nn as nn

# Move model on GPU if available
enable_cuda = True
device = torch.device('cuda' if torch.cuda.is_available() and enable_cuda else 'cpu')

class avg_pooling_cov_net(nn.Module):
    def __init__(self, dl = 100):
        super(avg_pooling_cov_net, self).__init__()
        if dl == 100:
            self.layer1 = nn.Sequential(
                nn.AdaptiveAvgPool2d((10, 10))
            )
        if dl == 81:
            self.layer1 = nn.Sequential(
                nn.AdaptiveAvgPool2d((9, 9))
            )
        if dl == 64:
            self.layer1 = nn.Sequential(
                nn.AdaptiveAvgPool2d((8, 8))
            )
        if dl == 49:
            self.layer1 = nn.Sequential(
                nn.AdaptiveAvgPool2d((7, 7)))
        if dl == 36:
            self.layer1 = nn.Sequential(
                nn.AdaptiveAvgPool2d((6, 6)))
        if dl == 25:
            self.layer1 = nn.Sequential(
                nn.AdaptiveAvgPool2d((5, 5)))
        if dl == 16:
            self.layer1 = nn.Sequential(
                nn.AdaptiveAvgPool2d((4, 4)))
        if dl == 9:
            self.layer1 = nn.Sequential(
                nn.AdaptiveAvgPool2d((3, 3)))
        self.flatten = nn.Flatten()

    def forward(self, x):
        out = self.layer1(x)
        out = self.flatten(out)
        return out

class CTDataset_ap(Dataset):
    def __init__(self, filepath, avg_pooling_model):
        self.x, self.y = torch.load(filepath, weights_only=False)
        self.x = self.x / 255.
        avg_pooling_model.eval()
        with torch.no_grad():
            self.x = avg_pooling_model(self.x)
        self.y = F.one_hot(self.y, num_classes=10).to(float)
        # self.y = self.y.to(float)
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]

torch.manual_seed(42)

for latent_space_dim in [9, 16, 25, 36, 49, 64, 81, 100]:

    avg_pooling_model = avg_pooling_cov_net(dl = latent_space_dim).to(device)
    train_ds = CTDataset_ap('./training.pt', avg_pooling_model)
    test_ds = CTDataset_ap('./test.pt', avg_pooling_model)

    torch.manual_seed(42)
    train_AE_set, train_cond_gen_set = torch.utils.data.random_split(train_ds, [30000, 30000])

    x_train, y_train = train_cond_gen_set[:]
    x_test, y_test = test_ds[:]

    x_train, y_train = x_train.cpu().detach().numpy(), y_train.cpu().detach().numpy()
    x_test, y_test = x_test.cpu().detach().numpy(), y_test.cpu().detach().numpy()

    classifiers = []
    param_dist = {
        "n_neighbors":
        [i for i in range(1, int(math.sqrt(x_train.shape[0])))]
    }
    random_search = RandomizedSearchCV(
        KNeighborsClassifier(),
        param_distributions=param_dist,
        n_iter=60,
        cv=5,
        n_jobs=-1,
        random_state=42 )
    classifiers.append(random_search.fit(x_train, y_train))

    # Assuming x_test and y_test are already defined
    test_accuracy = classifiers[0].score(x_test, y_test)
    print(f" latent_space_dim: {latent_space_dim}; Test Accuracy: {test_accuracy}") # 0.969 # 0.9695
    del train_ds, test_ds, x_train, y_train, x_test, y_test, avg_pooling_model, classifiers, random_search
    torch.cuda.empty_cache()

 latent_space_dim: 9; Test Accuracy: 0.6937
 latent_space_dim: 16; Test Accuracy: 0.7875
 latent_space_dim: 25; Test Accuracy: 0.897
 latent_space_dim: 36; Test Accuracy: 0.924
 latent_space_dim: 49; Test Accuracy: 0.9435
 latent_space_dim: 64; Test Accuracy: 0.9505
 latent_space_dim: 81; Test Accuracy: 0.9621
 latent_space_dim: 100; Test Accuracy: 0.9669


In [ ]:
# 0.6937, 0.7875, 0.897, 0.924, 0.9435, 0.9505, 0.9621, 0.9669